# Staff-Graded Matrix Completion

### Part(a)

In [41]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

# Import data as pandas df, convert to np.array
data = np.asarray(pd.read_csv('MovieReviewMat.csv'))

# Replace string 'NA' and blank with np.nan
data[data == 'NA'] = np.nan
data[data == ''] = np.nan

# Remove header rows and column, and ensure all values are float type
matrix = data[1:, 1:].astype(float)

# Find values
max_val = np.nanmax(matrix)
min_val = np.nanmin(matrix)
avg_val = np.nanmean(matrix)

print(f'Maximum Value: {max_val}')
print(f'Minimum Value: {min_val}')
print(f'Average Value: {avg_val}')


Maximum Value: 5.0
Minimum Value: 0.5
Average Value: 3.297080189685473


The maximum rating in the data is 5.0, and the minimum rating is 0.5

The average rating is approximately 3.3

### Part (b)

In [52]:
# Find genre labels and scores for individual
headers = data[0,1:]
scores_1460 = matrix[1460,:]
scores_43 = matrix[43,:]

def fivestar_review_count(headers, scores):
    # Concatenate all genre labels into one string
    string = str()
    for cell in headers:
        string += cell + '|'

    # Create function finds indeces of pipe character in concatenated string,
    # returns indeces in a list
    def find_letter(string, char):
        return [i for i, letter in enumerate(string) if letter == char]

    # Use function to find indeces list
    indeces = find_letter(string, '|')

    # Initialize empty list
    genre_list = list()

    # Fill empty list with non-unique genre names
    for i in range(len(indeces)):
        if i > 0:
            genre_list.append(string[indeces[i-1]+1:indeces[i]])
        else:
            genre_list.append(string[:indeces[i]])
                            
    # Unique-ify the genre list
    unique_genres = set(genre_list)
    unique_genres = list(unique_genres)
    unique_genres.sort()


    reviews_dict = {'Genre': unique_genres,
                    'Score': list(np.repeat(0, len(unique_genres)))}

    fivestar_reviews = pd.DataFrame(reviews_dict)

    i = 0
    for genre in fivestar_reviews['Genre']:
        j = 0
        for genre_combo in headers:
            if genre in genre_combo and scores[j] == 5:
                fivestar_reviews['Score'][i] += 1
            j += 1
        i += 1

    fivestar_reviews = fivestar_reviews.sort_values(by=['Score'], ascending=False)
    return fivestar_reviews


print(f'Five-star reviews for individual indexed at 1460, by genre:\n {fivestar_review_count(headers, scores_1460)}\n')
print(f'Five-star reviews for individual indexed at 43, by genre:\n {fivestar_review_count(headers, scores_43)}')

Five-star reviews for individual indexed at 1460:
                  Genre  Score
8                Drama     11
15             Romance      4
4             Children      2
1               Action      2
13             Musical      1
2            Adventure      1
18                 War      1
17            Thriller      1
6                Crime      1
16              Sci-Fi      1
9              Fantasy      1
0   (no genres listed)      0
14             Mystery      0
10           Film-Noir      0
12                IMAX      0
11              Horror      0
7          Documentary      0
5               Comedy      0
3            Animation      0
19             Western      0

Five-star reviews for individual indexed at 43:
                  Genre  Score
5               Comedy      5
8                Drama      2
2            Adventure      1
6                Crime      1
16              Sci-Fi      1
9              Fantasy      1
15             Romance      1
0   (no genres listed)      0

The first table above, which counts only five-star reviews, shows that indvidual 1460 tends to give the most five-star reviews to movies in the Drama genre.

The second table shows that individual 43 gives the most five-star reviews to movies in the Comedy genre.

